## Preliminaries

In [ ]:
library(ape)
library(ggtree)
library(phangorn)
library(Quartet)
library(adephylo)
library(ggplot2)
library(reshape2)
library(tidyverse)
library(gridExtra)
library(ggh4x)

#### Function to display face-to-face plot

In [ ]:
plot_cophylo <- function(tree1, tree2) {
    cophyloplot(ladderize(tree1), ladderize(tree2),
                assoc = matrix(rep(tree2$tip.label, 2), ncol = 2),
                length.line = 4, space = 28, gap = 3);
}

#### Function to display RF and Quartet metrics

In [ ]:
display_metrics <- function(tree, ref_tree) {
    splitStatuses <- SplitStatus(tree, ref_tree)
    quartetStatuses <- QuartetStatus(tree, ref_tree)
    # the least, the better (by distance)
    #print(paste("Robinson-Foulds: ", RawSymmetricDifference(splitStatuses, similarity = TRUE)))
    print(paste("RF (normalized)", RawSymmetricDifference(splitStatuses, similarity = TRUE) / splitStatuses[, 'N']))
    # the greater, the better (by similarity)
    print(paste("Quartet: ", SymmetricDifference(quartetStatuses, similarity = TRUE)))
}

## Plotting and comparison of trees

#### Loading reference tree

In [ ]:
ref_tree <- read.tree("dummy_tree.nwk")

In [ ]:
#>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
ngenes <- seq(from = 5000, to = 50000, by = 5000)
indels <- seq(from = 0.0, to = 0.9, by = 0.1)

rfs <- c()
for (i in indels) {
    display_metrics(read.tree(paste("rank/results_for_indel-rate/rankc_tree_indel-", i, ".nwk", sep = "")), ref_tree)
}
#for (i in indels) {
#    display_metrics(read.tree(paste("dcj/dcj_tree_indel-", i, ".nwk", sep = "")), ref_tree)
#}
#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<

#### Combinatios of simulator parameters

In [ ]:
confs <- c("GXIEz","GXIez","GXiEz","GXiez",
           "GxIEz","GxIez","GxiEz","Gxiez",
           "gXIEz","gXIez","gXiEz","gXiez",
           "gxIEz","gxIez","gxiEz","gxiez",
           "GXIEZ","GXIeZ","GXiEZ","GXieZ",
           "GxIEZ","GxIeZ","GxiEZ","GxieZ",
           "gXIEZ","gXIeZ","gXiEZ","gXieZ",
           "gxIEZ","gxIeZ","gxiEZ","gxieZ")

#### Functions to load metrics in dataframe

In [ ]:
get_quartet_metrics <- function (confs, dist, niters) {
    df <- setNames(data.frame(matrix(ncol = length(confs), nrow = 10)), confs)
    for (conf in confs) {
        quartets <- c()
        for (i in 1:niters) {
            tree <- read.tree(paste(dist, "/", dist, "_tree_", conf, "_iter", i, ".nwk", sep=""))
            quartetStatuses <- QuartetStatus(tree, ref_tree)
            quartet <- SymmetricDifference(quartetStatuses)
            quartets <- c(quartets, quartet)
        }
        df[conf] <- quartets
    }
    df
}

get_rf_metrics <- function (confs, dist, niters) {
    df <- setNames(data.frame(matrix(ncol = length(confs), nrow = 10)), confs)
    for (conf in confs) {
        rfs <- c()
        for (i in 1:niters) {
            tree <- read.tree(paste(dist, "/", dist, "_tree_", conf, "_iter", i, ".nwk", sep=""))
            splitStatuses <- SplitStatus(tree, ref_tree)
            rf <- RawSymmetricDifference(splitStatuses) / splitStatuses[, 'N']
            rfs <- c(rfs, rf)
        }
        df[conf] <- rfs
    }
    df
}

#### Functions to plot metrics and combinatios of simulator parameters as a grid

In [ ]:
plot_quartet <- function (df) {
    df %>%
    pivot_longer(everything()) %>%
    rename(quartet = value) %>%
    separate(name, into = c("gx", "ie", "z"), sep = c(2,4)) %>%
    ggplot(aes(x = "", y = quartet)) + 
        geom_boxplot() +
        facet_nested(gx ~ z + ie, scales = "free") +
        theme(axis.ticks.x = element_blank(), axis.title.x = element_blank())
}

plot_rf <- function (df) {
    df %>%
    pivot_longer(everything()) %>%
    rename(RF = value) %>%
    separate(name, into = c("gx", "ie", "z"), sep = c(2,4)) %>%
    ggplot(aes(x = "", y = RF)) + 
        geom_boxplot() +
        facet_nested(gx ~ z + ie, scales = "free") +
        theme(axis.ticks.x = element_blank(), axis.title.x = element_blank())
}

#### Rank vs. Reference

In [ ]:
#png("ref-vs-rank_quartet.png")
get_quartet_metrics(confs, "rank", 10) %>%
    plot_quartet()
#dev.off()

In [ ]:
#png("ref-vs-rank_rf.png")
get_rf_metrics(confs, "rank", 10) %>%
    plot_rf()
#dev.off()

#### Rank-Indel vs. Reference

In [ ]:
# no need

#### DCJ vs. Reference

In [ ]:
#png("ref-vs-dcj_quartet.png")
get_quartet_metrics(confs, "dcj", 10) %>%
    plot_quartet()
#dev.off()

In [ ]:
#png("ref-vs-dcj_rf.png")
get_rf_metrics(confs, "dcj", 10) %>%
    plot_rf()
#dev.off()